In [1]:
import sys
sys.path.insert(0,'..')

import pandas as pd
import numpy as np
import pickle 
import cv2
import math
import os

import config

In [2]:
ACTION_FILE = './data/actions.csv' # output from Lane_analysis notebook
LANE_FILE = './data/lanes_detections.csv' # output from detect_lanes notebook
OUTPUT_FILE = os.getcwd().split('/')[-1] # The output file that may be shared outside
OUTPUT_FILE = f'./data/{OUTPUT_FILE}_output.csv'
print ('Output file:', OUTPUT_FILE)


Output file: ./data/example_output.csv


In [3]:
VIDEO_FILE = pickle.load(open(f'{config.DATA_PATH}/videopath.p', 'rb'))

cap = cv2.VideoCapture(VIDEO_FILE) 
fps = cap.get(cv2.CAP_PROP_FPS)
print ('FPS:', fps)

FPS: 30.0


In [4]:
dfraw = pd.read_csv(LANE_FILE, header=None)
dfraw.columns = ['frame', 'lane', 'objectId', 'objectType', 'secMark', 
                 'xLeft', 'xRight', 'yTop', 'yBottom', 'lat', 'lon', 'speed', 'heading', 'elevation'] 
lane = dfraw.groupby('objectId').agg({
    'frame' : [np.min, np.max],  
    'objectType' : [lambda x:x.value_counts().index[0], 'mean'],
}).reset_index()
lane.columns = ['objectId', 'frame_start', 'frame_end', 'objectType', 'otMean']
lane.head()
lane = lane.merge(dfraw[['objectId', 'frame', 'lat', 'lon']], 
                  left_on = ['objectId', 'frame_start'],
                  right_on = ['objectId', 'frame']).drop('frame', axis=1)

lane = lane.merge(dfraw[['objectId', 'frame', 'lat', 'lon']], 
                  left_on = ['objectId', 'frame_end'],
                  right_on = ['objectId', 'frame']).drop('frame', axis=1)
lane = lane.rename(columns = {
    'lat_x' : 'lat_start',
    'lon_x' : 'lon_start',
    'lat_y' : 'lat_end',
    'lon_y' : 'lon_end',    
})

lane['time'] = (lane.frame_end - lane.frame_start) / fps
lane.head()

,objectId,frame_start,frame_end,objectType,otMean,lat_start,lon_start,lat_end,lon_end,time
0,10794154896_1686767151.553343,422,544,2,1.601626,45.742915,-122.660104,45.743897,-122.660418,4.066667
1,10796209552_1686767223.83911,636,892,2,2.000000,45.744255,-122.660892,45.741861,-122.660127,8.533333
2,10828166352_1686767259.0891688,741,741,2,2.000000,45.743902,-122.660422,45.743902,-122.660422,0.000000
3,10835342032_1686767021.614526,31,34,2,2.000000,45.741095,-122.657867,45.741122,-122.657892,0.100000
4,10835342032_1686767088.786319,200,200,1,1.000000,45.742213,-122.660033,45.742213,-122.660033,0.000000


In [5]:
def computeDistance(row):
    """
    Computes speeds between two points determined by
    (row.lat_start, row.lon_start), (row.lat_end, row.lon_end)
    Arguments:
        row - a structure (pd.Series) with defined the abovementioned
              lat/lon features
    Returns: 
        distance in meters
    """
    
    degrees_to_radians = math.pi / 180.0

    # phi = 90 - latitude
    phi1 = (90.0 - row.lat_start) * degrees_to_radians
    phi2 = (90.0 - row.lat_end) * degrees_to_radians

    # theta = longitude
    theta1 = row.lon_start * degrees_to_radians
    theta2 = row.lon_end * degrees_to_radians

    cos = (math.sin(phi1) * math.sin(phi2) * math.cos(theta1 - theta2) +
       math.cos(phi1) * math.cos(phi2))

    cos = max(-1, min(1, cos)) # in case of numerical problems

    ret = 6731000 * math.acos(cos) # mutliplied by earth radius in meters
    return ret

lane['dist'] = lane.apply(computeDistance, axis=1)
lane.head()

,objectId,frame_start,frame_end,objectType,otMean,lat_start,lon_start,lat_end,lon_end,time,dist
0,10794154896_1686767151.553343,422,544,2,1.601626,45.742915,-122.660104,45.743897,-122.660418,4.066667,118.209758
1,10796209552_1686767223.83911,636,892,2,2.000000,45.744255,-122.660892,45.741861,-122.660127,8.533333,288.147556
2,10828166352_1686767259.0891688,741,741,2,2.000000,45.743902,-122.660422,45.743902,-122.660422,0.000000,0.000000
3,10835342032_1686767021.614526,31,34,2,2.000000,45.741095,-122.657867,45.741122,-122.657892,0.100000,3.721917
4,10835342032_1686767088.786319,200,200,1,1.000000,45.742213,-122.660033,45.742213,-122.660033,0.000000,0.000000


In [6]:
lane['speed'] = lane.dist / lane.time * 2.237 # m/s -> MpH
lane = lane[lane.time > 0]
lane.head()

,objectId,frame_start,frame_end,objectType,otMean,lat_start,lon_start,lat_end,lon_end,time,dist,speed
0,10794154896_1686767151.553343,422,544,2,1.601626,45.742915,-122.660104,45.743897,-122.660418,4.066667,118.209758,65.025056
1,10796209552_1686767223.83911,636,892,2,2.000000,45.744255,-122.660892,45.741861,-122.660127,8.533333,288.147556,75.537431
3,10835342032_1686767021.614526,31,34,2,2.000000,45.741095,-122.657867,45.741122,-122.657892,0.100000,3.721917,83.259281
5,10835342032_1686767211.8253598,605,901,1,1.000000,45.744231,-122.660885,45.741587,-122.660103,9.866667,317.102862,71.894504
7,10835397008_1686767186.846976,532,538,2,2.000000,45.744271,-122.660813,45.744269,-122.660840,0.200000,2.224755,24.883888


In [7]:
df = pd.read_csv(ACTION_FILE)
df = df[['objectId', 'action', 'slowed', 'can_change']]
print (df.shape)
df = df.merge(lane, on='objectId')
print (df.shape)
df.head()

(27, 4)
(27, 15)


,objectId,action,slowed,can_change,frame_start,frame_end,objectType,otMean,lat_start,lon_start,lat_end,lon_end,time,dist,speed
0,11023984592_1686767012.5994189,noNeed,NaN,NaN,5,193,1,1.384615,45.742682,-122.660078,45.743949,-122.660468,6.266667,152.240795,54.345105
1,11023624464_1686767011.6896,noNeed,NaN,NaN,2,200,1,1.378788,45.742690,-122.660128,45.744027,-122.660544,6.600000,160.691226,54.464587
2,11023622160_1686767011.6895308,noNeed,NaN,NaN,2,226,1,1.468468,45.742562,-122.660102,45.743999,-122.660518,7.466667,172.252666,51.606591
3,11023627024_1686767011.6896682,noMoveOver,NaN,0.0,2,235,2,1.811966,45.742403,-122.660082,45.744085,-122.660600,7.766667,202.109926,58.212863
4,11023622736_1686767011.689548,noNeed,NaN,NaN,2,238,2,1.983122,45.742299,-122.659976,45.743946,-122.660465,7.866667,197.634985,56.200355


In [8]:
df = df[[
    'objectId', 'frame_start', 'frame_end', 'objectType', 
    #'speed', 
    'action', 'can_change', 'slowed']]
df.loc[df.can_change == 1, 'can_change'] = True
df.loc[df.can_change == 0, 'can_change'] = False
df.head()

,objectId,frame_start,frame_end,objectType,action,can_change,slowed
0,11023984592_1686767012.5994189,5,193,1,noNeed,NaN,NaN
1,11023624464_1686767011.6896,2,200,1,noNeed,NaN,NaN
2,11023622160_1686767011.6895308,2,226,1,noNeed,NaN,NaN
3,11023627024_1686767011.6896682,2,235,2,noMoveOver,False,NaN
4,11023622736_1686767011.689548,2,238,2,noNeed,NaN,NaN


In [9]:
df.to_csv(OUTPUT_FILE, index=False)

In [10]:
df.slowed.value_counts()

False    18
Name: slowed, dtype: int64